In [1]:
import numpy as np
from skimage import measure
from stl import mesh
import plotly.graph_objects as go  # for visualization
import os
import open3d as o3d
import tkinter as tk
from tkinter import filedialog
from time import perf_counter


import gyroid_utils
from gyroid_utils.utils import reload_all
reload_all()

working_path = os.getcwd()
print("Current working directory:", working_path)   

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
gyroid_utils: all modules reloaded
Current working directory: d:\COFO - HP\01 PhD research\202511 - gyroids\WP2 - diagnal gyroid


In [61]:
from OCC.Core.gp import gp_Pnt
from OCC.Core.BRepBuilderAPI import (
    BRepBuilderAPI_MakePolygon,
    BRepBuilderAPI_MakeWire,
    BRepBuilderAPI_MakeFace,
    BRepBuilderAPI_Sewing,
    BRepBuilderAPI_MakeSolid,
)
from OCC.Core.BRep import BRep_Builder
from OCC.Core.TopAbs import TopAbs_SHELL
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import TopoDS_Compound, topods_Shell, TopoDS_Shape
from OCC.Core.ShapeFix import ShapeFix_Shell
from OCC.Core.STEPControl import STEPControl_Writer, STEPControl_AsIs
from OCC.Core.IFSelect import IFSelect_RetDone


from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.ShapeUpgrade import ShapeUpgrade_UnifySameDomain

from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.Poly import Poly_Triangulation

from OCC.Core.ShapeUpgrade import ShapeUpgrade_ShapeDivideAngle
from OCC.Core.ShapeAnalysis import ShapeAnalysis_Shell
from OCC.Core.BRepCheck import BRepCheck_Shell

import sys
# Add the directory to the sys.path
sys.path.append('D:/COFO - HP/03 - Github/dvpt-at-EMPA/HappyBisons')
# Now you can import modules from that directory
import tools.loading_bar as loadingbar

from time import perf_counter

from OCC.Core.BRepCheck import BRepCheck_Analyzer

In [6]:
#==========================================================================================
#================================ load your STL file ======================================
#==========================================================================================
file_name = "gyroid-small"
file_name = "gyroid-diagnal-6"
verts, faces = gyroid_utils.io_ops.load_stl(working_path + '/' + file_name + '.stl')

# ------ visualize the quality of your mesh ----

triangle_areas = gyroid_utils.mesh_tools.calculate_triangle_areas(verts, faces)
gyroid_utils.viz.plot_histogram(triangle_areas)

[INFO] gyroid_utils: Loading STL file: d:\COFO - HP\01 PhD research\202511 - gyroids\WP2 - diagnal gyroid/gyroid-diagnal-6.stl
[INFO] gyroid_utils: Loaded STL successfully: 1491666 vertices, 499558 faces
[INFO] gyroid_utils: Calculating triangle areas…
[INFO] gyroid_utils: Triangle area computation complete.
[INFO] gyroid_utils: Plotting histogram for 499558 triangle areas


[INFO] gyroid_utils: Histogram displayed successfully.


In [ ]:
sewing_factor = 1000000
v_norm = np.sqrt(np.einsum('ij,ij->i', verts, verts))   # einstein sum : sum (e_i * e_i))
max_distance = np.max(v_norm)
print(f"max distance in the model: {max_distance}, sewing tolerance calculated at {max_distance/sewing_factor}")


print("Building B-Rep solid from triangles (can take time for dense meshes)...")
solid_normal, solid_simplified = gyroid_utils.occ_tools.faces_to_solid(
    vertices=verts,
    faces=faces,
    faces_area=triangle_areas,
    sew_tol=max_distance/sewing_factor
)

print("succesfully created solid, now saving result.")
step_path = f"{file_name}_normal.step"   # or ".stp"
gyroid_utils.io_ops.write_step(solid_normal, step_path)
print(f"STEP saved: {os.path.abspath(step_path)}")

step_path = f"{file_name}_simplified.step"   # or ".stp"
gyroid_utils.io_ops.write_step(solid_simplified, step_path)
print(f"STEP saved: {os.path.abspath(step_path)}")

print("Done.")

max distance in the model: 248.89544466709478, sewing tolerance calculated at 0.00024889544466709477
Building B-Rep solid from triangles (can take time for dense meshes)...
[INFO] gyroid_utils: Starting faces_to_solid()
[INFO] gyroid_utils: Number of faces: 499558
[INFO] gyroid_utils: Sew tolerance: 0.00024889544466709477
[INFO] gyroid_utils: Sewing triangles into shell…
[INFO] gyroid_utils: Building shell from 499558 triangles (sew_tol=0.00024889544466709477, area filter=on)
